<a href="https://colab.research.google.com/github/luasampaio/data-engineering/blob/main/7_ntb_AnalyticsWithPySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark
from pyspark.sql import SparkSession
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Create a SparkSession
spark = SparkSession.builder.appName("MySparkApp").getOrCreate()

# Verify the file path after mounting
!ls /content/drive/MyDrive/Maga/

# Update with the correct path if needed
file_path = "/content/drive/MyDrive/Maga/tab_venda_servicos.csv"

df = (
    spark.read.format("csv")
    .option("header", True)
    .load(file_path)
)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
json_transacoes_contas.txt	resultado_final.csv  servico.csv	     TESTE_LABS.sql
Labs_Maga_LucianaSampaio.ipynb	roubo_furto.csv      tab_venda_servicos.csv


In [2]:
df.printSchema()

root
 |-- data_venda: string (nullable = true)
 |-- cliente: string (nullable = true)
 |-- filial_venda: string (nullable = true)
 |-- servico: string (nullable = true)
 |-- valor_total: string (nullable = true)



In [3]:
df.show(5)

+----------+--------+------------+-------+-----------+
|data_venda| cliente|filial_venda|servico|valor_total|
+----------+--------+------------+-------+-----------+
|2022-10-22| 6309189|         256|      5|     232.27|
|2023-01-30|90585447|         256|      5|     425.13|
|2022-12-09|10733220|         256|      5|     122.67|
|2023-05-26| 6338901|         256|      5|     166.68|
|2023-06-30| 6278693|         256|      5|     509.95|
+----------+--------+------------+-------+-----------+
only showing top 5 rows



In [8]:
from pyspark.sql.functions import to_date, year, col

# Transformar a coluna 'data_venda' para o tipo Date
df = df.withColumn("data_venda", to_date("data_venda", "yyyy-MM-dd"))

# Converter a coluna 'valor_total' para o tipo Integer
df = df.withColumn("valor_total", col("valor_total").cast("int"))

# Criar a coluna 'ano' extraindo o ano de 'data_venda'
df = df.withColumn("ano", year("data_venda"))

# Exibir o esquema e as primeiras linhas
df.printSchema()
df.show(5)



root
 |-- data_venda: date (nullable = true)
 |-- cliente: string (nullable = true)
 |-- filial_venda: string (nullable = true)
 |-- servico: string (nullable = true)
 |-- valor_total: integer (nullable = true)
 |-- ano: integer (nullable = true)

+----------+--------+------------+-------+-----------+----+
|data_venda| cliente|filial_venda|servico|valor_total| ano|
+----------+--------+------------+-------+-----------+----+
|2022-10-22| 6309189|         256|      5|        232|2022|
|2023-01-30|90585447|         256|      5|        425|2023|
|2022-12-09|10733220|         256|      5|        122|2022|
|2023-05-26| 6338901|         256|      5|        166|2023|
|2023-06-30| 6278693|         256|      5|        509|2023|
+----------+--------+------------+-------+-----------+----+
only showing top 5 rows



In [9]:
df.printSchema()

root
 |-- data_venda: date (nullable = true)
 |-- cliente: string (nullable = true)
 |-- filial_venda: string (nullable = true)
 |-- servico: string (nullable = true)
 |-- valor_total: integer (nullable = true)
 |-- ano: integer (nullable = true)



In [6]:
#df = df.withColumnRenamed("data", "data_venda")
#df.printSchema()

root
 |-- data_venda: date (nullable = true)
 |-- cliente: string (nullable = true)
 |-- filial_venda: string (nullable = true)
 |-- servico: string (nullable = true)
 |-- valor_total: string (nullable = true)
 |-- ano: integer (nullable = true)



In [12]:
TotalAno = df.groupBy('ano').count().show()

+----+-------+
| ano|  count|
+----+-------+
|2023|1090543|
|2022|1238715|
+----+-------+



In [13]:
TotalAno = df.groupBy('ano').sum('valor_total').show()

+----+----------------+
| ano|sum(valor_total)|
+----+----------------+
|2023|       322582388|
|2022|       355024591|
+----+----------------+



Remover espaços nas colunas tipo string.

In [14]:
print("Total registros:",df.count())

Total registros: 2329258


**Serviços Mais Usados **

In [15]:
ranking = df.groupBy('servico').count().orderBy('count', ascending=False)
ranking.show()

+-------+-------+
|servico|  count|
+-------+-------+
|      5|1678748|
|     19| 330198|
|      7| 320312|
+-------+-------+



In [7]:
string_cols = [c for c, t in df.dtypes if t =='string']
from pyspark.sql import functions as f

for colname in string_cols:

    temp1 = df.withColumn(colname, f.trim(f.col(colname)))
    temp2 = temp1.withColumn(colname, f.ltrim(f.col(colname)))
    df = temp2.withColumn(colname, f.rtrim(f.col(colname)))